In [1]:
# !pip install openai


In [2]:
# !pip install newsapi-python

In [1]:
import openai
from getpass import getpass
openai.api_key = getpass()


In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=getpass())
from newspaper import fulltext
import requests

In [3]:
from dotenv import  dotenv_values

config = dotenv_values(".env")

from collections.abc import Mapping

import pyrebase
firebase = pyrebase.initialize_app(config)
database = firebase.database()  

In [5]:
headline = newsapi.get_everything(page=1,
                                  q="Technology"
                                  or "politics" 
                                  or "business"
                                  or "sports"
                                  or "entertainment"
                                  or "health"
                                  or "science"
                                  )
articles= headline['articles']
print(len(articles))

100


In [6]:
import time
headline = newsapi.get_everything(page=1,
                                  q="politics")
articles= headline['articles']
# print(articles)
pushData={}

for article in articles:
    
    try:
        url=article['url']
        html= requests.get(url).text
        text= fulltext(html)
        print("Text Pulled")
    
    except:
        print("Text Pull Failed")
        continue
    
    retries = 3    
    while retries > 0:    
        try: 

            openAIResult= openai.ChatCompletion.create(
                model= "gpt-3.5-turbo",
                messages=[
                    {"role":"system", "content": "You are a jounalist that summarizes news articals into short bullet points. "},
                    {"role":"user", "content": f"summarize the following article: {text} and return an python array of summarized bullet points. Also, Remove any bias from the article. Make the bullet points as factual as possible. "}
                ]
            )
            
            categoryResult= openai.ChatCompletion.create(
                model= "gpt-3.5-turbo",
                messages=[
                    {"role":"system", "content": "You are a jounalist that summarizes news articals into short bullet points. "},
                    {"role":"user", "content": f"Which news categories would  following article be in. It can have multiple category, return as string separated by commas and no and: {text}. Just give the categories. dont add any additional text "}
                ]
            )
            
           
            retries = -1
        except Exception as e:    
            if e: 
                print(e)   
                print('Timeout error, retrying...')    
                retries -= 1    
                time.sleep(5)    
            else:    
                raise e   
        
        
    summarized_text = openAIResult['choices'][0]['message']['content']
    # print(summarized_text)
    bullet_points = [point.strip() for point in summarized_text.split('\n') if point.strip()]
    article['bullet_points']= bullet_points
  
    
    
    categoryText= categoryResult['choices'][0]['message']['content'].split(',')
    article['category']= categoryText
    
    database.child("articles").push(article)



Text Pulled
Text Pulled
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Timeout error, retrying...
Text Pulled
Text Pulled
Text Pulled
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Timeout error, retrying...
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pull Failed
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Text Pulled
Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
Timeout error, retrying...
Text Pulled
